# Co-Channel and Adjacent Channel Interference Transforms
This notebook illustrates the two functional transforms that model co-channel interference and adjacent channel interference. The co-channel interference transform models interference as a configurable noise-like signal. The adjacent channel interference transform adds a similarly structured adjacent channel signal by modifying a copy of the original signal. Using an input high SNR tone signal, this notebook plots the frequency response within the entire sampled bandwidth for both transforms, highlighting the in-band and out-of-band responses. 

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.utils.defaults import default_dataset
import torchsig.transforms.functional as F
from torchsig.utils.dsp import low_pass

import numpy as np
import scipy.signal as sp

%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

Function for generating input tone signals in the test interference scenarios.

In [ ]:
def generate_tone_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband Tone Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 128.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["tone"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        noise_power_db=0.0,
        start_in_samples=0,
        seed=42,
    )
    signal = dataset.signal_generators[0]()

    # normalize, then scale data
    signal.data = F.normalize(data=signal.data, norm_order=2, flatten=False)
    signal.data = signal.data * scale
    return signal

Visualize the default filter design for the in-band and out-of-band channel frequency response.

In [ ]:
cutoff = 0.125
transition_bandwidth = 0.125
sample_rate = 1.0
lpf_weights = low_pass(
    cutoff=cutoff, transition_bandwidth=transition_bandwidth, sample_rate=sample_rate
)  # design lowpass filter

# plots
plt.style.use("dark_background")
fig, ax = plt.subplots()
spectrum, freqs, _ = ax.magnitude_spectrum(
    lpf_weights, Fs=sample_rate, scale="linear", sides="twosided", color="white"
)
ax.set_yscale("log")
ax.set_xlabel("Frequency")
ax.set_ylabel("Magnitude [log]");

Generate and frequency shift a new constant tone signal with calibrated power. Apply the co-channel interference transform to produce an output signal, then estimate the power and frequency of the original input component within the total aggregate signal. Verify the tone frequency is undisturbed by the transform.

In [ ]:
rng = np.random.default_rng(seed=42)

N = 16384
sample_rate = 2.0
power = 0.01
tone_freq = -0.04
filter_weights = low_pass(0.1, 0.15, 2.0)
tone_bb_data = generate_tone_signal(num_iq_samples=N, scale=1.0).data
tone_data = (
    tone_bb_data
    * np.exp(2j * np.pi * tone_freq * np.arange(N) / sample_rate)
    * np.sqrt(N)
)  # f0 = 0.2, total power = 1.0 W
est_power = np.sum(np.abs(tone_data) ** 2) / len(tone_data)
print("tone_data power: ", est_power)  # verify 1.0 W

data_co = F.cochannel_interference(
    data=tone_data,
    power=power,
    filter_weights=filter_weights,
    color="white",
    continuous=True,
)
est_power = np.sum(np.abs(data_co) ** 2) / len(data_co)
print("data_co power: ", est_power)  # verify 1.0 W + 1.0 W = 2.0 W (uncorrelated)

# verify peaks
D = np.abs(np.fft.fft(data_co, norm="ortho"))
freqs = np.fft.fftfreq(N) * sample_rate
peaks, _ = sp.find_peaks(D, height=1.0, distance=N / 2)
max_peak_ind = np.argmax(D[peaks])
print("peak freqs: ", freqs[peaks])

# plots
plt.style.use("dark_background")
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(
    tone_data, Fs=sample_rate, scale="linear", sides="twosided", color="white"
)
spectrum, freqs, _ = ax.magnitude_spectrum(
    data_co, Fs=sample_rate, scale="linear", sides="twosided", color="red"
)
ax.set_yscale("log")
ax.set_xlabel("Frequency")
ax.set_ylabel("Magnitude");

Generate and frequency shift a new constant tone signal with calibrated power. Apply the adjacent channel interference transform to produce an output signal, then estimate the power and frequency of the original input component within the total aggregate signal.

In [ ]:
rng = np.random.default_rng(seed=42)

tone_data = (
    tone_bb_data * np.exp(2j * np.pi * -0.11 * np.arange(N) / sample_rate) * np.sqrt(N)
)  # f0 = -0.11, total power = 1.0 W
sample_rate = 2.4

data_adj1 = F.adjacent_channel_interference(
    data=tone_data,
    power=2.0,
    sample_rate=sample_rate,
    center_frequency=-0.05,
    filter_weights=low_pass(0.25, 0.25, 2.4),
    phase_sigma=0.1,
    time_sigma=10.0,
    rng=rng,
)
est_power = np.sum(np.abs(data_adj1) ** 2) / len(data_adj1)
print(
    "output power: ", 10 ** (est_power / 10)
)  # verify 1.0 W + 1.0 W = 2.0 W (uncorrelated)

# plots
plt.style.use("dark_background")
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(
    tone_data, Fs=sample_rate, scale="linear", sides="twosided", color="white"
)
spectrum, freqs, _ = ax.magnitude_spectrum(
    data_adj1, Fs=sample_rate, scale="linear", sides="twosided", color="red"
)
ax.set_yscale("log")
ax.set_xlabel("Frequency")
ax.set_ylabel("Magnitude");